In [2]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats

In [3]:
# Specify the years of interest
years = [1990, 1995, 2000] + list(range(2005, 2021))

# Create an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

for year in years:
    raster_path = f'/Users/andrewzimmer/Documents/Montana State - Postdoc/Research/Zimmer - Urban Demography : AQ : Heat/data/aq/no2/{year}_final_1km.tif'
    shapefile_path = '/Users/andrewzimmer/Documents/Montana State - Postdoc/Research/Zimmer - Urban Demography : AQ : Heat/data/UCDB/ghs_ucdb.shp'
    shp = gpd.read_file(shapefile_path)
    stats = zonal_stats(shp, raster_path, stats="mean")
    stats_zonal = pd.DataFrame(stats)
    stats_zonal['fid'] = shp['fid']
    stats_zonal['Name'] = shp['UC_NM_MN']
    stats_zonal.rename({'mean': f"mean_{year}"}, axis=1, inplace=True)
    print(f"Completed processing data for year {year}")

    # Merge the data on 'fid' and 'Name' fields
    if merged_data.empty:
        merged_data = stats_zonal
    else:
        merged_data = pd.merge(merged_data, stats_zonal, on=['fid', 'Name'], how='outer')

# Display the merged data
merged_data

Completed processing data for year 1990
Completed processing data for year 1995
Completed processing data for year 2000
Completed processing data for year 2005
Completed processing data for year 2006
Completed processing data for year 2007


In [ ]:
# Melt the data frame
melted_df = pd.melt(merged_data, id_vars=['ID', 'Name'], var_name='year', value_name='NO2')

# Extract year from the 'year' column
melted_df['year'] = melted_df['year'].str.extract('(\d+)')

# Drop the 'name' column
melted_df = melted_df.drop(columns=['Name'])

# Check it looks ok
melted_df

In [ ]:
# Export the file
merged_data.to_csv('/Users/andrewzimmer/Documents/Montana State - Postdoc/Research/Zimmer - Urban Demography : AQ : Heat/data/aq/no2/ucdb-no2-extracted.csv', index = False)